# Building a Simple Information Retrieval System using BM25 and GPT-3 and evaluated in the CISI collection

<a target="_blank" href="https://colab.research.google.com/github/tcvieira/bm25-exercise-report/blob/main/notebook.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Imports

In [1]:
import numpy as np 
import pandas as pd
import random
import pickle

# Load Dataset

In [2]:
## Download dataset

!wget https://ir.dcs.gla.ac.uk/resources/test_collections/cisi/cisi.tar.gz -P content/
!tar -xzvf 'content/cisi.tar.gz' -C content/

--2023-02-21 18:54:04--  https://ir.dcs.gla.ac.uk/resources/test_collections/cisi/cisi.tar.gz
Resolving ir.dcs.gla.ac.uk (ir.dcs.gla.ac.uk)... 130.209.240.253
Connecting to ir.dcs.gla.ac.uk (ir.dcs.gla.ac.uk)|130.209.240.253|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 775144 (757K) [application/gzip]
Saving to: ‘content/cisi.tar.gz’

cisi.tar.gz         100%[===================>] 756,98K   609KB/s    in 1,2s    

2023-02-21 18:54:07 (609 KB/s) - ‘content/cisi.tar.gz’ saved [775144/775144]

x CISI.ALL
x CISI.BLN
x CISI.QRY
x CISI.REL


In [3]:
### Processing DOCUMENTS
doc_set = {}
doc_id = ""
doc_text = ""
with open('content/CISI.ALL') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
doc_count = 0
for l in lines:
    if l.startswith(".I"):
        doc_id = int(l.split(" ")[1].strip())-1
    elif l.startswith(".X"):
        doc_set[doc_id] = doc_text.lstrip(" ")
        doc_id = ""
        doc_text = ""
    else:
        doc_text += l.strip()[3:] + " " # The first 3 characters of a line can be ignored.    

        
### Processing QUERIES
with open('content/CISI.QRY') as f:
    lines = ""
    for l in f.readlines():
        lines += "\n" + l.strip() if l.startswith(".") else " " + l.strip()
    lines = lines.lstrip("\n").split("\n")
    
qry_set = {}
qry_id = ""
for l in lines:
    if l.startswith(".I"):
        qry_id = int(l.split(" ")[1].strip()) -1
    elif l.startswith(".W"):
        qry_set[qry_id] = l.strip()[3:]
        qry_id = ""

### Processing QRELS
rel_set = {}
with open('content/CISI.REL') as f:
    for l in f.readlines():
        qry_id = int(l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[0]) -1
        doc_id = int(l.lstrip(" ").strip("\n").split("\t")[0].split(" ")[-1])-1
        if qry_id in rel_set:
            rel_set[qry_id].append(doc_id)
        else:
            rel_set[qry_id] = []
            rel_set[qry_id].append(doc_id)

## Simple dataset EDA

In [4]:
print(f'{len(doc_set)} documents')
print(f'{len(qry_set)} queries') 
print(f'{len(rel_set)} mappings ground truth')

number_rel_docs = [len(value) for key, value in rel_set.items()]
print(f'Average of {np.mean(number_rel_docs):.2f} documents per query')
print(f'minimum of {np.min(number_rel_docs)} relevant documents per query')

qry_no_docs = np.setdiff1d(list(qry_set.keys()),list(rel_set.keys()))
print(f'There are {len(qry_no_docs)} queries without relevant documents: {qry_no_docs}')

1460 documents
112 queries
76 mappings ground truth
Average of 40.97 documents per query
minimum of 1 relevant documents per query
There are 36 queries without relevant documents: [ 35  37  39  46  47  50  52  58  59  62  63  67  69  71  72  73  74  76
  77  79  82  84  85  86  87  88  90  92  93 102 104 105 106 107 109 111]


Some pairs of query and relevant documents

In [5]:
random.seed(42)
idx = random.sample(list(rel_set.keys()),1)[0]

print(f'Query ID {idx}: {qry_set[idx]}')
print()

rel_docs = rel_set[idx]
print(f'Documents relevants to Query ID {idx}: {rel_docs}')
print()

sample_document_idx = random.sample(rel_docs,1)[0]
print(f'Document ID {sample_document_idx}: {doc_set[sample_document_idx]}')

Query ID 14: How much do information retrieval and dissemination systems, as well as automated libraries, cost? Are they worth it to the researcher and to industry?

Documents relevants to Query ID 14: [17, 26, 35, 48, 55, 58, 66, 73, 82, 125, 157, 163, 166, 191, 213, 221, 222, 249, 280, 291, 294, 298, 306, 330, 335, 337, 347, 364, 365, 366, 367, 371, 380, 445, 457, 464, 465, 481, 489, 490, 494, 496, 506, 519, 527, 590, 593, 622, 628, 638, 689, 719, 722, 723, 726, 727, 730, 778, 821, 833, 838, 847, 848, 864, 871, 896, 1099, 1160, 1247, 1304, 1352, 1357, 1362, 1365, 1367, 1370, 1371, 1373, 1374, 1375, 1376, 1409]

Document ID 48: Adaptive Information Dissemination Sage, C.R. Anderson, R.R. Fitzwater, D.R. Computer dissemination of information offers significant advantages over manual dissemination because the computer can use strategies that are impractical and in some cases impossible for a human.. This paper describes the Ames Laboratory Selective Dissemination of Information system w

# Preprocessing

In [6]:
%pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [7]:
import nltk
nltk.download('punkt')
nltk.download('stopwords')

stemmer = nltk.stem.PorterStemmer()
stop_words = nltk.corpus.stopwords.words('english')

[nltk_data] Downloading package punkt to /Users/tcvieira/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tcvieira/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [8]:
def preprocess_string(txt, remove_stop=True, do_stem=True, to_lower=True):
    """
    Return a preprocessed tokenized text.
    
    Args:
        txt (str): original text to process
        remove_stop (boolean): to remove or not stop words (common words)
        do_stem (boolean): to do or not stemming (suffixes and prefixes removal)
        to_lower (boolean): remove or not capital letters.
        
    Returns:
        Return a preprocessed tokenized text.
    """      
    if to_lower:
        txt = txt.lower()
    tokens = nltk.tokenize.word_tokenize(txt)
    
    if remove_stop:
        tokens = [tk for tk in tokens if tk not in stop_words]
    if do_stem:
        tokens = [stemmer.stem(tk) for tk in tokens]
    return tokens

In [9]:
corpus = list(doc_set.values())
tokenized_corpus = [doc.split(" ") for doc in corpus]
tokenized_corpus_with_preprocessing = [preprocess_string(doc) for doc in corpus]

# Evaluating Metric MRR@10 

In [10]:
##Source: https://gist.github.com/bwhite/3726239
def mean_reciprocal_rank(bool_results, k=10):
    """Score is reciprocal of the rank of the first relevant item
    First element is 'rank 1'.  Relevance is binary (nonzero is relevant).
    Example from http://en.wikipedia.org/wiki/Mean_reciprocal_rank
    >>> rs = [[0, 0, 1], [0, 1, 0], [1, 0, 0]]
    >>> mean_reciprocal_rank(rs)
    0.61111111111111105

    Args:
        rs: Iterator of relevance scores (list or numpy) in rank order
            (first element is the first item)
    Returns:
        Mean reciprocal rank
    """
    bool_results = (np.atleast_1d(r[:k]).nonzero()[0] for r in bool_results)
    return np.mean([1. / (r[0] + 1) if r.size else 0. for r in bool_results])

mean_reciprocal_rank([[0, 0, 1], [0, 1, 0], [1, 0, 0]])

0.611111111111111

In [11]:
def results_from_query(qry_id, bm25, pre_process=False):
    """Return an ordered array of relevant documents returned by query_id

    Args:
        qry_id (int): id of query on dataset
        bm25 (object): indexed corpus

    Returns:
        boolean sorted relevance array of documents
    """    
    query = qry_set[qry_id]
    rel_docs = []
    if qry_id in rel_set:
        rel_docs = rel_set[qry_id]
    if pre_process:
        tokenized_query = preprocess_string(query)
    else:
        tokenized_query = query.split(" ")
    scores = bm25.get_scores(tokenized_query)
    most_relevant_documents = np.argsort(-np.asarray(scores))
    masked_relevance_results = np.zeros(most_relevant_documents.shape)
  
    masked_relevance_results[rel_docs] = 1
    sorted_masked_relevance_results = np.take(masked_relevance_results, most_relevant_documents)
    
    return sorted_masked_relevance_results

# Simple BM25 implementation

In [12]:
import math

# Implementation from https://en.wikipedia.org/wiki/Okapi_BM25
class BM25Simple(object):
    PARAM_K1 = 1.2
    PARAM_B = 0.75
    EPSILON = 0.25

    def __init__(self, corpus):
        self.corpus_size = len(corpus)
        self.dl = [float(len(d)) for d in corpus]
        self.avgdl = sum(self.dl) / self.corpus_size
        self.corpus = corpus
        self.f = []
        self.df = {}
        self.idf = {}
        self.average_idf = 0
        self._initialize()

    def _initialize(self):
        for document in self.corpus:
            frequencies = {}
            for word in document:
                if word not in frequencies:
                    frequencies[word] = 0
                frequencies[word] += 1
            self.f.append(frequencies)

            for word, freq in frequencies.items():
                if word not in self.df:
                    self.df[word] = 0
                self.df[word] += 1

        for word, freq in self.df.items():
            self.idf[word] = math.log(self.corpus_size - freq + 0.5) - math.log(freq + 0.5)

        self.average_idf = sum(map(lambda k: float(self.idf[k]), self.idf.keys())) / len(self.idf.keys())

    def _get_score(self, document, index):
        score = 0
        for word in document:
            if word not in self.f[index]:
                continue
            idf = self.idf[word] if self.idf[word] >= 0 else self.EPSILON * self.average_idf
            score += (idf * self.f[index][word] * (self.PARAM_K1 + 1)
                      / (self.f[index][word] + self.PARAM_K1 * (1 - self.PARAM_B + self.PARAM_B * self.dl[index] / self.avgdl)))
        return score

    def _get_scores(self, document):
        scores = []
        for index in range(self.corpus_size):
            score = self._get_score(document, index)
            scores.append(score)
        return scores

    def get_scores(self, query, k=None):
        """Returns the `scores` of most relevant documents according to `query`"""
        result = [(index, score) for index, score in enumerate(self._get_scores(query))]
        result.sort(key=lambda x: x[1], reverse=True)
        _, scores = zip(*result)
        return scores
    
    def get_top_n(self, query, corpus, n=20):
        """Returns the `indexes` most relevant documents according to `query`"""
        result = [(index, score) for index, score in enumerate(self._get_scores(query))]
        result.sort(key=lambda x: x[1], reverse=True)
        indexes, _ = zip(*result)
        return [corpus[i] for i in indexes[:n]]


## Indexing all documents from dataset with no preprocessing

In [13]:
bm25_simple = BM25Simple(tokenized_corpus) # initialize bm25 model
print(bm25_simple.avgdl)

132.96301369863014


## Select one query

In [14]:
random.seed(42)
idx = random.sample(list(rel_set.keys()),1)[0]
query = qry_set[idx] #get query text
rel_docs = rel_set[idx] #get relevant documents

In [15]:
print(query)
print(rel_docs)

How much do information retrieval and dissemination systems, as well as automated libraries, cost? Are they worth it to the researcher and to industry?
[17, 26, 35, 48, 55, 58, 66, 73, 82, 125, 157, 163, 166, 191, 213, 221, 222, 249, 280, 291, 294, 298, 306, 330, 335, 337, 347, 364, 365, 366, 367, 371, 380, 445, 457, 464, 465, 481, 489, 490, 494, 496, 506, 519, 527, 590, 593, 622, 628, 638, 689, 719, 722, 723, 726, 727, 730, 778, 821, 833, 838, 847, 848, 864, 871, 896, 1099, 1160, 1247, 1304, 1352, 1357, 1362, 1365, 1367, 1370, 1371, 1373, 1374, 1375, 1376, 1409]


## Process query and get scores for each indexed document

In [16]:
tokenized_query = query.split(" ")
most_relevant_documents = bm25_simple.get_scores(tokenized_query)
print(most_relevant_documents[:20])

(27.69059098299087, 26.089249721540284, 25.57380016889922, 25.324928895166078, 25.043902089468702, 24.18612909012977, 23.999872624571303, 23.860219369082273, 23.852044277609302, 23.69679134227739, 23.60121321726178, 23.499630788232857, 23.32750348503679, 23.256338826046168, 23.242008880343104, 23.147564214065753, 22.710861164464262, 22.651688787691786, 22.610927246354244, 22.4616504889854)


## Evaluating single query

In [17]:
# create the boolean mask for the relevant documents to calculate MRR@10
masked_relevance_results = np.zeros(len(most_relevant_documents))
masked_relevance_results[rel_docs] = 1
print(masked_relevance_results[:20])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [18]:
sorted_masked_relevance_results = np.take(masked_relevance_results, most_relevant_documents)
print(sorted_masked_relevance_results[:20])

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [19]:
# Calculate MRR@10 for document
print(mean_reciprocal_rank([sorted_masked_relevance_results]))

0.5


## Evaluating all queries

In [20]:
results = [results_from_query(qry_id, bm25_simple) for qry_id in list(qry_set.keys())]
print(f'MRR@10 {mean_reciprocal_rank(results):.4f}')

MRR@10 0.0629


## Save model

In [21]:
# file_name = "models/BM25_simple.pkl"
# with open(file_name, 'wb') as file:
#     pickle.dump(bm25_simple, file)

In [22]:
# with open(file_name, 'rb') as file:
#     model: BM25Simple = pickle.load(file)
#     print(model.corpus_size)

1460


# BM25Okapi implementation

In [23]:
%pip install rank_bm25
from rank_bm25 import BM25Okapi

Note: you may need to restart the kernel to use updated packages.


## Indexing all documents from dataset with no preprocessing

In [24]:
# Index all documents using BM25
print(tokenized_corpus[0])
bm25 = BM25Okapi(tokenized_corpus)

['18', 'Editions', 'of', 'the', 'Dewey', 'Decimal', 'Classifications', 'Comaromi,', 'J.P.', 'The', 'present', 'study', 'is', 'a', 'history', 'of', 'the', 'DEWEY', 'Decimal', 'Classification.', '', 'The', 'first', 'edition', 'of', 'the', 'DDC', 'was', 'published', 'in', '1876,', 'the', 'eighteenth', 'edition', 'in', '1971,', 'and', 'future', 'editions', 'will', 'continue', 'to', 'appear', 'as', 'needed.', '', 'In', 'spite', 'of', 'the', "DDC's", 'long', 'and', 'healthy', 'life,', 'however,', 'its', 'full', 'story', 'has', 'never', 'been', 'told.', '', 'There', 'have', 'been', 'biographies', 'of', 'Dewey', 'that', 'briefly', 'describe', 'his', 'system,', 'but', 'this', 'is', 'the', 'first', 'attempt', 'to', 'provide', 'a', 'detailed', 'history', 'of', 'the', 'work', 'that', 'more', 'than', 'any', 'other', 'has', 'spurred', 'the', 'growth', 'of', 'librarianship', 'in', 'this', 'country', 'and', 'abroad.', '']


## Select one query

In [25]:
random.seed(42)
idx = random.sample(list(rel_set.keys()),1)[0]
query = qry_set[idx] #get query text
rel_docs = rel_set[idx] #get relevant documents

## Process query and get scores for each indexed document

In [26]:
tokenized_query = query.split(" ")
print(f'query: {query}')
print(f'Relevant documents IDs: {rel_docs}')

scores = bm25.get_scores(tokenized_query)
print(scores, len(scores), len(doc_set))

query: How much do information retrieval and dissemination systems, as well as automated libraries, cost? Are they worth it to the researcher and to industry?
Relevant documents IDs: [17, 26, 35, 48, 55, 58, 66, 73, 82, 125, 157, 163, 166, 191, 213, 221, 222, 249, 280, 291, 294, 298, 306, 330, 335, 337, 347, 364, 365, 366, 367, 371, 380, 445, 457, 464, 465, 481, 489, 490, 494, 496, 506, 519, 527, 590, 593, 622, 628, 638, 689, 719, 722, 723, 726, 727, 730, 778, 821, 833, 838, 847, 848, 864, 871, 896, 1099, 1160, 1247, 1304, 1352, 1357, 1362, 1365, 1367, 1370, 1371, 1373, 1374, 1375, 1376, 1409]
[13.84492303 12.72656528 17.02184487 ... 12.72737224 14.26660278
 14.10298505] 1460 1460


## Evaluating single query

In [27]:
# argsort gives the indexes of values in increasing order, so we input with the negative values of scores
most_relevant_documents = np.argsort(-scores)
print(most_relevant_documents[:20])

[ 655  593  254  663  767  176  292  514  363  139   23  313 1026  460
 1447 1236  825  326 1276   27]


In [28]:
# create the boolean mask for the relevant documents to calculate MRR@10
masked_relevance_results = np.zeros(most_relevant_documents.shape)
masked_relevance_results[rel_docs] = 1
print(masked_relevance_results[:20])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [29]:
sorted_masked_relevance_results = np.take(masked_relevance_results, most_relevant_documents)
print(sorted_masked_relevance_results[:20])

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [30]:
# Calculate MRR@10 for document
print(mean_reciprocal_rank([sorted_masked_relevance_results]))

0.5


## Evaluating all queries

In [31]:
results = [results_from_query(qry_id, bm25) for qry_id in list(qry_set.keys())]
print(f'MRR@10 {mean_reciprocal_rank(results):.4f}')

MRR@10 0.3146


## Save model

In [32]:
# file_name = "models/BM25Okapi.pkl"
# with open(file_name, 'wb') as file:
#     pickle.dump(bm25, file)

In [33]:
# with open(file_name, 'rb') as file:
#     model: BM25Okapi = pickle.load(file)
#     print(model.corpus_size)

1460


# BM25+ implementation

In [34]:
%pip install rank_bm25
from rank_bm25 import BM25Plus

Note: you may need to restart the kernel to use updated packages.


## Indexing all documents from dataset

In [35]:
# Index all documents using BM25
corpus = list(doc_set.values())
tokenized_corpus = [doc.split(" ") for doc in corpus]
print(tokenized_corpus[0])
bm25_plus = BM25Plus(tokenized_corpus)

['18', 'Editions', 'of', 'the', 'Dewey', 'Decimal', 'Classifications', 'Comaromi,', 'J.P.', 'The', 'present', 'study', 'is', 'a', 'history', 'of', 'the', 'DEWEY', 'Decimal', 'Classification.', '', 'The', 'first', 'edition', 'of', 'the', 'DDC', 'was', 'published', 'in', '1876,', 'the', 'eighteenth', 'edition', 'in', '1971,', 'and', 'future', 'editions', 'will', 'continue', 'to', 'appear', 'as', 'needed.', '', 'In', 'spite', 'of', 'the', "DDC's", 'long', 'and', 'healthy', 'life,', 'however,', 'its', 'full', 'story', 'has', 'never', 'been', 'told.', '', 'There', 'have', 'been', 'biographies', 'of', 'Dewey', 'that', 'briefly', 'describe', 'his', 'system,', 'but', 'this', 'is', 'the', 'first', 'attempt', 'to', 'provide', 'a', 'detailed', 'history', 'of', 'the', 'work', 'that', 'more', 'than', 'any', 'other', 'has', 'spurred', 'the', 'growth', 'of', 'librarianship', 'in', 'this', 'country', 'and', 'abroad.', '']


## Select one query

In [36]:
random.seed(42)
idx = random.sample(list(rel_set.keys()),1)[0]
query = qry_set[idx] #get query text
rel_docs = rel_set[idx] #get relevant documents

## Process query and get scores for each indexed document

In [37]:
tokenized_query = query.split(" ")
print(f'query: {query}')
print(f'Relevant documents IDs: {rel_docs}')

scores = bm25.get_scores(tokenized_query)
print(scores, len(scores), len(doc_set))

query: How much do information retrieval and dissemination systems, as well as automated libraries, cost? Are they worth it to the researcher and to industry?
Relevant documents IDs: [17, 26, 35, 48, 55, 58, 66, 73, 82, 125, 157, 163, 166, 191, 213, 221, 222, 249, 280, 291, 294, 298, 306, 330, 335, 337, 347, 364, 365, 366, 367, 371, 380, 445, 457, 464, 465, 481, 489, 490, 494, 496, 506, 519, 527, 590, 593, 622, 628, 638, 689, 719, 722, 723, 726, 727, 730, 778, 821, 833, 838, 847, 848, 864, 871, 896, 1099, 1160, 1247, 1304, 1352, 1357, 1362, 1365, 1367, 1370, 1371, 1373, 1374, 1375, 1376, 1409]
[13.84492303 12.72656528 17.02184487 ... 12.72737224 14.26660278
 14.10298505] 1460 1460


## Evaluating single query

In [38]:
# argsort gives the indexes of values in increasing order, so we input with the negative values of scores
most_relevant_documents = np.argsort(-scores)
print(most_relevant_documents[:20])

[ 655  593  254  663  767  176  292  514  363  139   23  313 1026  460
 1447 1236  825  326 1276   27]


In [39]:
# create the boolean mask for the relevant documents to calculate MRR@10
masked_relevance_results = np.zeros(most_relevant_documents.shape)
masked_relevance_results[rel_docs] = 1
print(masked_relevance_results[:20])

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [40]:
sorted_masked_relevance_results = np.take(masked_relevance_results, most_relevant_documents)
print(sorted_masked_relevance_results[:20])

[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [41]:
# Calculate MRR@10 for document
print(mean_reciprocal_rank([sorted_masked_relevance_results]))

0.5


## Evaluating all queries

In [42]:
results = [results_from_query(qry_id, bm25_plus) for qry_id in list(qry_set.keys())]
print(f'MRR@10 {mean_reciprocal_rank(results):.4f}')

MRR@10 0.3334


## Save model

In [43]:
# file_name = "models/BM25Plus.pkl"
# with open(file_name, 'wb') as file:
#     pickle.dump(bm25, file)

In [44]:
# with open(file_name, 'rb') as file:
#     model: BM25Plus = pickle.load(file)
#     print(model.corpus_size)

1460


# Evaluation with preprocessing

## BM25Simple

In [45]:
results = [results_from_query(qry_id, bm25_simple, pre_process=True) for qry_id in list(qry_set.keys())]
print(f'MRR@10 {mean_reciprocal_rank(results):.4f}')

MRR@10 0.0629


## BM25OKapi

In [46]:
results = [results_from_query(qry_id, bm25, pre_process=True) for qry_id in list(qry_set.keys())]
print(f'MRR@10 {mean_reciprocal_rank(results):.4f}')

MRR@10 0.1475


## BM25+

In [47]:
results = [results_from_query(qry_id, bm25_plus, pre_process=True) for qry_id in list(qry_set.keys())]
print(f'MRR@10 {mean_reciprocal_rank(results):.4f}')

MRR@10 0.1487


# Requirements

In [48]:
print('\n'.join(f'{m.__name__}=={m.__version__}' for m in globals().values() if getattr(m, '__version__', None)))

numpy==1.22.1
pandas==1.4.3
nltk==3.8.1
